# KSU Fertility Calculator
__Name:__ Bryan Rutter  
__Semester:__ Spring 2020  
__Project Area:__ Soil Fertility/Agronomy  

## Objective:

Create function(s) to automate the calculation of lime and fertilizer application rates based on soil test results from the KSRE Soil Testing Laboratory.

For more details regarding the interpretation of soil tests visit https://bookstore.ksre.ksu.edu/pubs/mf2586.pdf

In [1]:
# Load required modules
import pandas as pd
import numpy as np
import timeit

In [2]:
# Import example data
STL_data = pd.read_csv('./example_data.csv') # Import soil test data and save as dataframe

In [3]:
# Declare custom rounding functions
def round5(x):
    """
    Function that rounds inputs to the nearest 5. This is used to round fertilizer application rates.
    """
    return 5 * round(x/5)

def round50(x):
    """
    Function that rounds input to the nearest 50. This is used in lime application rate
    """
    return 50 * round(x/50)

In [4]:
# Declare fertilizer Recommendation function
def KSU_fertility_recs(df, save = 0):
    """
    A function that calculates NPK fertilizer and lime application rates based on equations from KSU Fact Sheet MF-2586. The output is intended to serve as a "check" for lab employees regarding application rate calculations performed by STRS.
    
    Arguments:
    df = a DataFrame containing soil test information generated by the KSU Soil Testing Lab's "Soil Test Reporting System (STRS)".
    save = Specify whether a copy of the output data should be saved to file. Save = 0 (default) results in no file being saved, setting "save" equal to any other number will save a copy of the dataframe as a csv file named "example_output.csv"
    
    Author: Bryan Rutter
    Creation Date: 4/6/2020
    Author email: rutter@ksu.edu 
    Lab email: soiltesting@ksu.edu
    """
    # define parameters
    sample_depth = df["SampleDepth"]
    incorp_depth = np.where(df["Tillage"] == "No-Till", 3, 6)
    
    ## Code below calculates fertility recommendations for various crops
    # Calculate recommended nitrogen (N) application rates
    df.loc[df["Crop"] == "Corn", "N_KSU_Rec"] = round5(df["YieldGoal"] * 1.6 - df["OM"] * 20 - df["NO3"] * 0.3 * sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Corn N-Rec (lbs N / acre)
    df.loc[(df["Crop"] == "Soybeans") | (df["Crop"] == "Alfalfa") | (df["Crop"] == "Clover"), "N_KSU_Rec"] = 0 # Soybean and Established Alfalfa/Clover N-Rec (lbs N / acre)
    df.loc[(df["Crop"] == "Alfalfa (New)") | (df["Crop"] == "Clover (New)"), "N_KSU_Rec"] = 20 # New Alfalfa and Clover N-Rec (lbs N / acre)
    df.loc[df["Crop"] == "Sorghum Grain (Milo)", "N_KSU_Rec"] = round5(1.6 * df["YieldGoal"] - df["OM"] * 20 - df["NO3"] * 0.3 * sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Sorghum/Milo N-Rec (lbs N / acre)
    df.loc[df["Crop"] == "Wheat", "N_KSU_Rec"] = round5(2.4 * df["YieldGoal"] - df["OM"] * 10 - df["NO3"] * 0.3 * sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Wheat N-Rec (lbs N / acre)
    df.loc[df["Crop"] == "Sunflower", "N_KSU_Rec"] = round5(0.075 * df["YieldGoal"] - df["OM"] * 20 - df["NO3"] * 0.3 * sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Sunflower N-Rec (lbs N / acre)
    df.loc[df["Crop"] == "Oats", "N_KSU_Rec"] = round5(1.3 * df["YieldGoal"] - df["OM"] * 10 - df["NO3"] * 0.3* sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Oats N-Rec (lbs N / acre)
    df.loc[df["Crop"] == "Corn Silage", "N_KSU_Rec"] = round5(10.67 * df["YieldGoal"] - df["OM"] * 20 - df["NO3"] * 0.3* sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Corn Silage N-Rec (lbs N / acre)
    df.loc[df["Crop"] == "Sorghum Silage", "N_KSU_Rec"] = round5(10.67 * df["YieldGoal"] - df["OM"] * 20 - df["NO3"] * 0.3* sample_depth - (df["PrevCrop"] == "Soybeans")*40) # Sorghum Silage N-Rec (lbs N / acre)
    df.loc[(df["Crop"] == "Brome") | (df["Crop"] == "Fescue") | (df["Crop"] == "Bermuda"), "N_KSU_Rec"] = round5(40 * df["YieldGoal"] - df["NO3"] * 0.3* sample_depth) # Forage grasses N-Rec (lbs N / acre)
    df.loc[(df["Crop"] == "Brome (New)") | (df["Crop"] == "Fescue (New)") | (df["Crop"] == "Bermuda (New)"), "N_KSU_Rec"] = 20 # New seeding forage grasses N-Rec (lbs N / acre)

    # Calculate recommended phosphorus (P) application rates
    df.loc[df["Crop"] == "Corn", "P2O5_KSU_Rec"] = round5(50 + df["YieldGoal"] * 0.2 + df["P"] * -2.5 + df["YieldGoal"] * df["P"] * - 0.01) # Corn P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Corn Silage", "P2O5_KSU_Rec"] = round5(56 + df["YieldGoal"] * 1.12 + df["P"] * -2.8 + df["YieldGoal"] * df["P"] * - 0.056) # Corn Silage P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Wheat", "P2O5_KSU_Rec"] = round5(46 + df["YieldGoal"] * 0.42 + df["P"] * -2.3 + df["YieldGoal"] * df["P"] * - 0.021) # Wheat P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Sorghum Grain (Milo)", "P2O5_KSU_Rec"] = round5(50 + df["YieldGoal"] * 0.2 + df["P"]*-2.5 + df["YieldGoal"] * df["P"] * - 0.01) # Sorghum/Milo P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Sorghum Silage", "P2O5_KSU_Rec"] = round5(48 + df["YieldGoal"] * 1.19 + df["P"] * -2.38 + df["YieldGoal"] * df["P"] * -0.0594) # Sorghum Silage P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Soybeans", "P2O5_KSU_Rec"] = round5(56 + df["YieldGoal"] * 0.51 + df["P"] * -2.8 + df["YieldGoal"] * df["P"] * - 0.0257) # Soybean P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Sunflowers", "P2O5_KSU_Rec"] = round5(42 + df["YieldGoal"] * .01 + df["P"] * -2.1 + df["YieldGoal"] * df["P"] * - 0.0005) # Sunflowers P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Oats", "P2O5_KSU_Rec"] = round5(47 + df["YieldGoal"] * .25 + df["P"] * -2.3 + df["YieldGoal"] * df["P"] * - 0.013) # Oats P2O5 Rec (lbs P2O5 / acre)
    df.loc[(df["Crop"] == "Brome") | (df["Crop"] == "Fescue"), "P2O5_KSU_Rec"] = round5(44 + df["YieldGoal"] * 6.3 + df["P"] * -2.2 + df["YieldGoal"] * df["P"] * - 0.315) # Brome/Fescue P2O5 Rec (lbs P2O5 / acre)
    df.loc[df["Crop"] == "Bermuda", "P2O5_KSU_Rec"] = round5(64 + df["YieldGoal"] * 5.3 + df["P"] * -2.56 + df["YieldGoal"] * df["P"] * - 0.21) # Bermuda P2O5 Rec (lbs P2O5 / acre)
    df.loc[(df["Crop"] == "Alfalfa") | (df["Crop"] == "Clover"), "P2O5_KSU_Rec"] = round5(73 + df["YieldGoal"] * 4.56 + df["P"] * -2.92 + df["YieldGoal"] * df["P"] * - 0.18) # Alfalfa P2O5 Rec (lbs P2O5 / acre)
    df.loc[(df["Crop"] == "Alfalfa (New)") | (df["Crop"] == "Clover (New)"), "P2O5_KSU_Rec"] = round5(84 + df["YieldGoal"] * 12 + df["P"] * - 3.37 + df["YieldGoal"] * df["P"] * - 0.48) # New seeding Alfalfa/Clover P2O5 Rec (lbs P2O5 / acre)
    
    # Calculate recommended potassium (K) application rates
    df.loc[df["Crop"] == "Corn", "K2O_KSU_Rec"] = round5(73 + df["YieldGoal"] *0.21 + df["K"] * - 0.565 + df["YieldGoal"] * df["K"] * - 0.0016) # Corn K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Wheat", "K2O_KSU_Rec"] = round5(62 + df["YieldGoal"] *0.24 + df["K"] * - 0.48 + df["YieldGoal"] * df["K"] * - 0.0018) # Wheat K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Sorghum Grain (Milo)", "K2O_KSU_Rec"] = round5(80 + df["YieldGoal"] * 0.17 + df["K"] * -0.616 + df["YieldGoal"] * df["K"] * - 0.0013) # Sorghum/Milo K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Soybeans", "K2O_KSU_Rec"] = round5(60 + df["YieldGoal"] * 0.628 + df["K"] * - 0.46 + df["YieldGoal"] * df["K"] * - 0.0048) # Soybean K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Sunflowers", "K2O_KSU_Rec"] = round5(80 + df["YieldGoal"] * 0.008 + df["K"] * - 0.622 + df["YieldGoal"] * df["K"] * - 0.00006) # Sunflowers K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Oats", "K2O_KSU_Rec"] = round5(62 + df["YieldGoal"] * 0.221 + df["K"] * - 0.48 + df["YieldGoal"] * df["K"] * - 0.0017) # Sunflowers K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Corn Silage", "K2O_KSU_Rec"] = round5(74 + df["YieldGoal"] * 1.50 + df["K"] * - 0.567 + df["YieldGoal"] * df["K"] * - 0.0115) # Corn Silage K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Sorghum Silage", "K2O_KSU_Rec"] = round5(73 + df["YieldGoal"] * 1.8 + df["K"] * - 0.56 + df["YieldGoal"] * df["K"] * - 0.0139) # Sorghum Silage K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Brome", "K2O_KSU_Rec"] = round5(41 + df["YieldGoal"] * 5.85 + df["K"] * - 0.315 + df["YieldGoal"] * df["K"] * - 0.045) # Brome K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Fescue", "K2O_KSU_Rec"] = round5(41 + df["YieldGoal"] * 5.85 + df["K"] * - 0.315 + df["YieldGoal"] * df["K"] * - 0.045) # Fescue K2O Rec (lbs K2O / acre)
    df.loc[df["Crop"] == "Bermuda", "K2O_KSU_Rec"] = round5(75 + df["YieldGoal"] * 6.25 + df["K"] * - 0.5 + df["YieldGoal"] * df["K"] * - 0.042) # Bermuda K2O Rec (lbs K2O / acre)
    df.loc[(df["Crop"] == "Alfalfa") | (df["Crop"] == "Clover"), "K2O_KSU_Rec"] = round5(84 + df["YieldGoal"] * 5.24 + df["K"] * - 0.56 + df["YieldGoal"] * df["K"] * - 0.035) # Alfalfa/clover K2O Rec (lbs K2O / acre)
    df.loc[(df["Crop"] == "Alfalfa (New)") | (df["Crop"] == "Clover (New)"), "K2O_KSU_Rec"] = round5(105 + df["YieldGoal"] * 15 + df["K"] * - 0.7 + df["YieldGoal"] * df["K"] * - 0.1) # New seeding alfalfa/clover K2O Rec (lbs K2O / acre)
    
    # Calculate lime application rates
    df["Lime_rec_KSU"] = np.where(df["pH"] < 5.8, round50((14100 - 3540*df["BUFpH"] + 224*df["BUFpH"]*df["BUFpH"])*incorp_depth), 0)
    
    # Select which columns to return
    output_cols = ["OrderNo","LabNo","Crop", "LimeApp",  "Lime_rec_KSU", "NApp1", "N_KSU_Rec","P2O5App", "P2O5_KSU_Rec", "K2OApp", "K2O_KSU_Rec" ]
    df_out = df[output_cols]
    
    # Negative values don't make sense for application rates; replace them with zero
    int_cols = ["LimeApp", "Lime_rec_KSU", "NApp1", "N_KSU_Rec","P2O5App", "P2O5_KSU_Rec", "K2OApp", "K2O_KSU_Rec"]
    df_out[int_cols] = df_out[int_cols].mask(df_out[int_cols]<0, 0)
    
    # Save output to csv file
    if save != 0:
        df_out.to_csv("example_output.csv")
    else:
        None
    
    return df_out

In [5]:
KSU_fertility_recs(STL_data)

C:\Users\rutter1\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,OrderNo,LabNo,Crop,LimeApp,Lime_rec_KSU,NApp1,N_KSU_Rec,P2O5App,P2O5_KSU_Rec,K2OApp,K2O_KSU_Rec
0,26634,8238,Corn,2350,2300.0,50,50.0,0,0.0,0,0.0
1,26634,8238,Sorghum Grain (Milo),2350,2300.0,5,0.0,0,0.0,0,0.0
2,26634,8239,Corn,0,0.0,0,0.0,0,-0.0,0,0.0
3,26634,8239,Sorghum Grain (Milo),0,0.0,0,0.0,0,-0.0,0,0.0
4,26634,8240,Corn,2350,2300.0,55,55.0,10,10.0,0,0.0
5,26634,8240,Sorghum Grain (Milo),2350,2300.0,10,10.0,10,10.0,0,0.0
6,26634,8241,Sorghum Grain (Milo),2150,2050.0,130,130.0,10,10.0,0,0.0
7,26634,8241,Soybeans,2150,2050.0,0,0.0,10,10.0,0,0.0
8,26634,8242,Corn,0,0.0,100,100.0,70,70.0,0,0.0
9,26634,8242,Sorghum Grain (Milo),0,0.0,50,50.0,60,65.0,0,0.0
